In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras.models import Sequential
from bayes_opt import BayesianOptimization
import keras
from keras import Model
from keras.layers import Dense, Flatten, Embedding, Conv1D, LSTM, GRU, MaxPooling1D, Dropout, Input, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.metrics import AUC
from keras.layers import concatenate
from keras.constraints import max_norm
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_fscore_support
from keras.regularizers import l1, l2, l1_l2
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
import tensorflow as tf
import time

In [ ]:
# Bij problemen
# !pip install --ignore-installed PyYAML

!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.3 MB/s 
     |████████████████████████████████| 7.6 MB 43.8 MB/s 
     |████████████████████████████████| 182 kB 69.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.3 MB/s 


# Tokenizers

In [ ]:
from transformers import BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def bert_encode(data_input,maximum_length) :
  input_ids = []
  token_type_ids = []
  attention_masks = []

  for text in data_input:
      encoded = bert_tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=maximum_length,
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,        
      )      
      input_ids.append(encoded['input_ids'])
      token_type_ids.append(encoded['token_type_ids'])
      attention_masks.append(encoded['attention_mask'])

  return np.array(input_ids),np.array(token_type_ids),np.array(attention_masks)  

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

# Transformers

In [ ]:
import tensorflow as tf
import transformers 
from transformers import AutoTokenizer
from sklearn.metrics import roc_auc_score


early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=2, mode="max")

def bayes_opt_BERT_Hybrid(xtmeta, xtwords, yt, xvmeta, xvwords, yv,
                                 num_random_points, num_iterations):
    
    inputs_train, tokens_train, attentions_train = bert_encode(xtwords,input_length)
    inputs_val, tokens_val, attentions_val = bert_encode(xvwords,input_length)

    # parameters for cnn that do NOT have to be saved
    maximum_epochs = 10

    # parameters that change for each iteration that must be saved
    list_early_stop_epochs = []
    list_validation_auc = []
    list_saved_model_name = []
     
    start_time_total = time.time()
    
    def bert_hybrid_function(dense_meta, dense_text):
         
        model_name =  'BERT_' + str(np.random.uniform(0,1,))[2:9]

        # variable parameters
        dict_params = {'dense_meta':16*int(dense_meta),
                      'dense_text':32*int(dense_text)}
        
        # channel 1 (Meta Channel)
        inputs1 = Input(shape=(input_length_meta,))
        mlp1 = Dense(units=dict_params['dense_meta'], activation='relu')(inputs1)
       
        # channel 2 (Text Channel: 'Transformer')
        encoder = transformers.TFBertModel.from_pretrained('bert-base-uncased')

        inputs2 = Input(shape=(input_length,), dtype=tf.int32)
        
        token_type_ids = Input(shape=(input_length,), dtype=tf.int32)
        attention_mask = Input(shape=(input_length,), dtype=tf.int32)
        embedding = encoder(
            inputs2, token_type_ids=token_type_ids, attention_mask=attention_mask
        )[0]

        mlp2 = Dense(units=dict_params['dense_text'], activation='relu')(embedding)
        flat2 = Flatten()(mlp2)

        # merge
        merged = concatenate([mlp1, flat2])

        # last dense layer
        final_hidden = Dense(64, activation='sigmoid')(merged)
        # output layer 
        outputs = Dense(1, activation='sigmoid')(final_hidden)
        # instantiate and compile model
        hybrid_model = Model(inputs=[inputs1,inputs2, token_type_ids, attention_mask], outputs=outputs) 
        
        hybrid_model.layers[3].trainable = False

        hybrid_model.compile(optimizer='adam', loss='binary_crossentropy')
        # fit the model
        h = hybrid_model.fit(x=[xtmeta,inputs_train, tokens_train, attentions_train], y=yt, epochs=maximum_epochs, verbose=0, callbacks=[early_stopping_monitor], 
                          validation_data=([xvmeta,inputs_val, tokens_val, attentions_val], yv))
        list_early_stop_epochs.append(len(h.history['val_loss']))
         
        predictions_val = hybrid_model.predict([xvmeta, inputs_val, tokens_val, attentions_val])
        validation_auc = roc_auc_score(yv, predictions_val)   
        list_validation_auc.append(validation_auc)
        list_saved_model_name.append(model_name)
         
        # bayes opt is a maximization algorithm, so it can simply maximize the validation auc (for other metric, where smaller is better: 1-validation_score)
        bayes_opt_score = validation_auc
         
        return bayes_opt_score 
    
    optimizer = BayesianOptimization(f=bert_hybrid_function,
                pbounds={'dense_meta':(1, 4.001),  
                         'dense_text':(1, 4.001)},
                         verbose=2)
     
    optimizer.maximize(init_points=num_random_points, n_iter=num_iterations)
     
    print('nbest result:', optimizer.max)
     
    elapsed_time_total = (time.time()-start_time_total)/60
    print('\n\ntotal elapsed time =',elapsed_time_total,' minutes')
     
    # optimizer.res is a list of dicts
    list_dfs = []
    counter = 0
    for result in optimizer.res:
        df_temp = pd.DataFrame.from_dict(data=result['params'], orient='index',
                                         columns=['trial' + str(counter)]).T
        df_temp['bayes opt error'] = result['target']
         
        df_temp['epochs'] = list_early_stop_epochs[counter] 
        df_temp['validation_auc'] = list_validation_auc[counter]
        df_temp['model_name'] = list_saved_model_name[counter]
         
        list_dfs.append(df_temp)
         
        counter = counter + 1
         
    df_results = pd.concat(list_dfs, axis=0)
    return df_results

def bert_hybrid_predict(xtmeta, xtwords, yt, xvmeta, xvwords, dense_meta, dense_text, final_epochs):
      
    inputs_train, tokens_train, attentions_train = bert_encode(xtwords,input_length)
    inputs_val, tokens_val, attentions_val = bert_encode(xvwords,input_length)
    # variable parameters
    dict_params = {'dense_meta':16*int(dense_meta),
                  'dense_text':32*int(dense_text)}
    
    # channel 1 (Meta Channel)
    inputs1 = Input(shape=(input_length_meta,))
    mlp1 = Dense(units=dict_params['dense_meta'], activation='relu')(inputs1)
    
    # channel 2 (Text Channel: 'Transformer')
    encoder = transformers.TFBertModel.from_pretrained('bert-base-uncased')

    inputs2 = Input(shape=(input_length,), dtype=tf.int32)
    
    token_type_ids = Input(shape=(input_length,), dtype=tf.int32)
    attention_mask = Input(shape=(input_length,), dtype=tf.int32)
    embedding = encoder(
        inputs2, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    mlp2 = Dense(units=dict_params['dense_text'], activation='relu')(embedding)
    flat2 = Flatten()(mlp2)

    # merge
    merged = concatenate([mlp1, flat2])

    # last dense layer
    final_hidden = Dense(64, activation='sigmoid')(merged)
    # output layer 
    outputs = Dense(1, activation='sigmoid')(final_hidden)
    # instantiate and compile model
    hybrid_model = Model(inputs=[inputs1,inputs2, token_type_ids, attention_mask], outputs=outputs) 
    
    hybrid_model.layers[3].trainable = False

    hybrid_model.compile(optimizer='adam', loss='binary_crossentropy')
    # fit the model
    h = hybrid_model.fit(x=[xtmeta,inputs_train, tokens_train, attentions_train], y=yt, epochs=final_epochs, verbose=0)
      
    predictions_test = hybrid_model.predict([xvmeta, inputs_val, tokens_val, attentions_val])

    return hybrid_model, predictions_test

# Interpretation Functions

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

def explain_instance_with_data(neighborhood_data,
                               labels_column,
                               weights,
                               used_feat):
    
    easy_model = Ridge(alpha=1, fit_intercept=True)
    scaler = StandardScaler()
    selected_data = neighborhood_data[neighborhood_data.columns.intersection(used_feat)]
    selected_data = scaler.fit_transform(selected_data)
    easy_model.fit(selected_data, labels_column, sample_weight=weights)
    prediction_score = easy_model.score(selected_data, labels_column, sample_weight=weights)
    
    return (sorted(zip(used_feat, easy_model.coef_[0]),
                   key=lambda x: np.abs(x[1]), reverse=True),
            prediction_score)

def feat_selection(data, labels, weights, num_features):
    clf = Ridge(alpha=0.01, fit_intercept=True)
    scaler = StandardScaler()
    df = scaler.fit_transform(data)
    clf.fit(df, labels, sample_weight=weights)
    coef = abs(clf.coef_)
    used_features = data.columns[coef.argsort()][0][0:num_features]
    return(used_features)

# Main Loop

In [ ]:
import pickle
import os 
os.chdir('/content/drive/My Drive/Rumour Identification COVID-19/Modellen/')
dataset = 'covid'


final_stability_metrics = []
import itertools
import functools
import operator
import collections
from sklearn.metrics.pairwise import pairwise_kernels

dat_num = 0
dataset ='covid'
if True:
  dat_num += 1
  permutations = os.listdir('/content/drive/My Drive/Rumour Identification COVID-19/Modellen/')
  permutations = [each_string.lower() for each_string in permutations]
  permutations = [el for el in permutations if dataset in el]
  permutations = [el for el in permutations if 'permuted' in el][0]
  with open(permutations,'rb') as f:
              save_dict = pickle.load(f)

  if True:
    for fold in range(1,11):
    
      trainsmall = pd.read_csv('/content/drive/My Drive/Rumour Identification COVID-19/DataProject/'+str(dataset)+'_trainsmall'+str(fold))
      val = pd.read_csv('/content/drive/My Drive/Rumour Identification COVID-19/DataProject/'+str(dataset)+'_val'+str(fold))
      train = pd.read_csv('/content/drive/My Drive/Rumour Identification COVID-19/DataProject/'+str(dataset)+'_train'+str(fold))
      test = pd.read_csv('/content/drive/My Drive/Rumour Identification COVID-19/DataProject/'+str(dataset)+'_test'+str(fold))

      trainsmall = trainsmall.drop(columns = 'Unnamed: 0')
      val = val.drop(columns = 'Unnamed: 0')

      x_trainBig_text = train['text']
      x_test_text = test['text']
      x_train_text = trainsmall['text']
      x_val_text = val['text']

      x_trainBig_meta = train.iloc[:,204:242]
      x_test_meta = test.iloc[:,204:242]
      x_train_meta = trainsmall.iloc[:,204:242]
      x_val_meta = val.iloc[:,204:242]

      y_trainBig = train.iloc[:,1]
      y_test = test.iloc[:,1]
      y_val = val.iloc[:,1]
      y_train = trainsmall.iloc[:,1]

      x_trainBig_meta= np.asarray(x_trainBig_meta).astype(np.float32)
      x_test_meta = np.asarray(x_test_meta).astype(np.float32) 
      x_train_meta = np.asarray(x_train_meta).astype(np.float32)
      x_val_meta = np.asarray(x_val_meta).astype(np.float32)

      y_trainBig= np.asarray(y_trainBig).astype(np.float32)
      y_test = np.asarray(y_test).astype(np.float32) 
      y_train = np.asarray(y_train).astype(np.float32)
      y_val = np.asarray(y_val).astype(np.float32)

      input_length_meta = x_train_meta.shape[1]
      input_length = max([len(text) for text in x_train_text])

      results_bert = bayes_opt_BERT_Hybrid(x_train_meta, x_train_text, y_train, x_val_meta, x_val_text, y_val, 5, 5)
      if len(results_bert['epochs'].unique())!=1:
        results_bert = results_bert[results_bert['epochs']!=1]
      best_trial = results_bert[results_bert['validation_auc']==results_bert['validation_auc'].max()]
      hybrid_bert_model, y_pred = bert_hybrid_predict(x_trainBig_meta, x_trainBig_text, y_trainBig, x_test_meta,  x_test_text, best_trial['dense_meta'], best_trial['dense_text'],  int(best_trial['epochs']))
      
      uitkomsten = []        
      model = hybrid_bert_model
      
      if dataset=='covid':
        train['cleaned_text'] = np.where(train['Unnamed: 0']==1629, 'empty text with url', train['cleaned_text'])

      for i in range(train.shape[0]): 
          true_obs = train.iloc[i,:]
          key = true_obs['Unnamed: 0']
          if (key==1629 and dataset=='covid'):
            print('faulty obs')
          else:  
            permutations_obs = save_dict[key]
            tussen = permutations_obs.drop(columns = 'id')
            interpretable_x = tussen.filter(regex='^\D')
            x_lime = tussen[tussen.columns.drop(list(tussen.filter(regex='bow_')))]
            interpretable_x = interpretable_x.drop(columns = 'text')
            x_lime = x_lime.drop(columns = [str(el) for el in range(200)])
            x_lime = x_lime[x_lime.columns.drop(list(x_lime.filter(regex='\.')))]
            print(i)
            x_lime_text = x_lime['text']
            inputs_lime, tokens_lime, attentions_lime = bert_encode(x_lime_text, input_length)
            x_lime_meta = x_lime.drop(columns = 'text')

            true_obs1 = true_obs[x_lime_meta.columns]
            true_obs2 = true_obs[tussen.filter(regex='\.').columns]
            true_obs = pd.concat([true_obs1, true_obs2], axis = 0)
            x_lime_meta = np.asarray(x_lime_meta).astype(np.float32)

            y_lime = model.predict([x_lime_meta, inputs_lime, tokens_lime, attentions_lime ], batch_size=32)
            x_lime = tussen[true_obs.index]

            kernel_values = pairwise_kernels(true_obs.values.reshape(1, -1), x_lime, metric='linear')[0]
            if min(kernel_values)<0:
              kernel_values = kernel_values-(min(kernel_values)-1)
            used_features = feat_selection(interpretable_x, y_lime, kernel_values, 20)
            uitkomst = explain_instance_with_data(interpretable_x, y_lime, kernel_values, used_features)
            uitkomsten.append(uitkomst)
          if i%100==0:
            fidelity = np.mean([el[1] for el in uitkomsten])
            var_importances = [dict(el[0]) for el in uitkomsten]
            res = []
            for el in var_importances:
              res.append({key: abs(val) for key, val in el.items()})
            result = dict(functools.reduce(operator.add, map(collections.Counter, res)))

            final_stability_metrics.append([fold, 'BERT', result, fidelity])
            with open('ZekerNietVerliezen_BERT_'+dataset+'.pickle', 'wb') as f:
              pickle.dump(final_stability_metrics, f)
      fidelity = np.mean([el[1] for el in uitkomsten])
      var_importances = [dict(el[0]) for el in uitkomsten]
      res = []
      for el in var_importances:
          res.append({key: abs(val) for key, val in el.items()})
      result = dict(functools.reduce(operator.add, map(collections.Counter, res)))

      final_stability_metrics.append([fold, 'BERT', result, fidelity])
  
      with open('ZekerNietVerliezen_BERT_'+dataset+'.pickle', 'wb') as f:
        pickle.dump(final_stability_metrics, f) 
  
  with open('ZekerNietVerliezen_BERT_'+dataset+'.pickle', 'wb') as f:
    pickle.dump(final_stability_metrics, f) 

63


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


|   iter    |  target   | dense_... | dense_... |
-------------------------------------------------


Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


32/32 [==============================] - 87s 3s/step
695


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


32/32 [==============================] - 87s 3s/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


In [5]:
dataset = 'covid'
import os
import pickle

os.chdir('/content/drive/My Drive/Rumour Identification COVID-19/Modellen/')

with open('ZekerNietVerliezen_BERT_'+dataset+'.pickle', 'rb') as f:
    final_stability_metrics = pickle.load(f) 

In [8]:
import numpy as np

def jaccard_spearman(top_N, used_set):
    top = [list(metric[2].keys())[0:top_N] for metric in used_set]
    jaccard = []
    spearman = []
    for lst in top:
        for other_lst in top:
            st = set(lst)
            other_st = set(other_lst)
            jaccard.append(len(st.intersection(other_st))/len(st.union(other_st)))
            new_lst = lst + list(other_st.difference(st))
            new_other = other_lst + list(st.difference(other_st))
            zipbObj = zip(new_lst, range(len(new_lst)))
            dict1 = dict(zipbObj)
            zipbObj = zip(new_other, range(len(new_other)))
            dict2 = dict(zipbObj)
            res = {key: abs(dict2[key] - dict1.get(key, 0))  for key in dict2.keys()}
            rank_differences = list(res.values())
            summed_squared_differences = np.sum([diff ** 2 for diff in rank_differences])
            spearman.append(1-((6*summed_squared_differences)/((len(rank_differences)**3)-len(rank_differences))))
    result_one = np.mean(jaccard)
    result_two = np.mean(spearman)
    return(result_one, result_two)

def jaccard(top_N, used_set):
    top = [list(metric[2].keys())[0:top_N] for metric in used_set]
    jaccard = []
    for lst in top:
        for other_lst in top:
            st = set(lst)
            other_st = set(other_lst)
            jaccard.append(len(st.intersection(other_st))/len(st.union(other_st)))
    result_one = np.mean(jaccard)
    return(result_one)

def return_scores(used_set):
    avg_fidelity = np.mean([metric[3] for metric in used_set])
    jaccard20 = jaccard(20, used_set)
    jaccard10 = jaccard(10, used_set)
    jaccard5 = jaccard(5, used_set)
    return(avg_fidelity, jaccard20, jaccard10, jaccard5)

final_scores = return_scores(final_stability_metrics)

final_scores

(0.07437784819468328,
 0.37941063122119134,
 0.2535438921425142,
 0.20147761384263105)